In [1]:
DATA_NAME = 'syntheticMRI2D-coronal' 
TRANSFORM = 'wavelet-vertical'
CHANNEL = ''

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_vertical_wavelet_coronal_syntheticMRI2D.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        if len(group_data_map[group]) < 100:
            X[:, i] = np.zeros(bootstrap_size)
        else:
            X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.0,6627953.83,132.46,-1.39,-22.29,1.82,-1.06,0.01
4,0.0,132.46,2351329.63,47.15,6.09,9.32,-1.37,-0.04
5,0.0,-1.39,47.15,577948.63,2.10,-1.95,0.99,0.03
6,0.0,-22.29,6.09,2.10,141859.82,-1.16,-0.22,0.10
7,0.0,1.82,9.32,-1.95,-1.16,20325.35,0.09,0.00
8,0.0,-1.06,-1.37,0.99,-0.22,0.09,1470.36,0.00
9,0.0,0.01,-0.04,0.03,0.10,0.00,0.00,65.57


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

/var/folders/0c/5hhs4r6n7pnc4jcl295d7j_c0000gn/T/ipykernel_19910/980427090.py:1: RuntimeWarning: invalid value encountered in divide
  corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))


,2,3,4,5,6,7,8,9
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,1.00000,0.00003,-0.00000,-0.00002,0.00000,-0.00001,0.00000
4,NaN,0.00003,1.00000,0.00004,0.00001,0.00004,-0.00002,-0.00000
5,NaN,-0.00000,0.00004,1.00000,0.00001,-0.00002,0.00003,0.00000
6,NaN,-0.00002,0.00001,0.00001,1.00000,-0.00002,-0.00002,0.00003
7,NaN,0.00000,0.00004,-0.00002,-0.00002,1.00000,0.00002,0.00000
8,NaN,-0.00001,-0.00002,0.00003,-0.00002,0.00002,1.00000,0.00000
9,NaN,0.00000,-0.00000,0.00000,0.00003,0.00000,0.00000,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

202.03206623358338

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[5.50812865e+12 6.76331498e+11 3.97914592e+10 2.29676407e+09
 4.42271425e+07 2.05880490e+05 3.06954669e+02 0.00000000e+00]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.0,0.998261,-0.057578,-0.012237,-0.002982,-0.000426,-0.000031,-0.000001
1,-0.0,0.056952,0.997344,-0.044231,-0.010138,-0.001443,-0.000105,-0.000005
2,-0.0,0.014541,0.042866,0.997566,-0.052569,-0.007002,-0.000504,-0.000023
3,0.0,0.004300,0.012138,0.051741,0.997890,-0.036948,-0.002605,-0.000115
4,-0.0,0.000768,0.002160,0.008816,0.036439,0.998997,-0.024332,-0.001078
5,0.0,0.000074,0.000209,0.000846,0.003456,0.024185,0.999451,-0.022358
6,-0.0,0.000005,0.000014,0.000057,0.000231,0.001613,0.022325,0.999749
7,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[1.0,
 0.943048487930146,
 0.9571344760528071,
 0.9482591800033845,
 0.963560882079383,
 0.975814769351956,
 0.9776749432845914,
 1.0]